In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import torch
from torch import nn
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
import pandas as pd

In [ ]:
class ToTensor(object):
    def __call__(self, sample):
        window, labels = sample['window'], sample['labels']
        return {'window': torch.tensor(window.values).unsqueeze(0).to(torch.float32), 
                'labels': torch.tensor(labels).to(torch.float32)}#.values)}

class Normalize(object):
    def __call__(self, sample):
        window, labels = sample['window'], sample['labels']
        return {'window': F.normalize(torch.abs(window)), 
                'labels': labels}

class EEGDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.csv_file = pd.read_csv(csv_file, delimiter='\t', header=None)
        self.transform = transform
    
    def __len__(self):
        return len(self.csv_file)
    
    def __getitem__(self, idx):
        # if idx >= len(self.csv_file) / 1250:
        #     return

        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        window = self.csv_file.iloc[idx*1250:idx*1250+1250, 7]
        labels = [self.csv_file.iloc[idx*1250, 32]]#:idx*1250+1250, 32] # remove brackets for timestep prediction
        sample = {'window': window, 'labels': labels}

        if self.transform:
            sample = self.transform(sample)
        
        return sample

In [ ]:
eeg_dataset = EEGDataset(csv_file='./test.csv', transform=transforms.Compose([ToTensor(),
                                                                              Normalize()]))
eeg_dataset = torch.utils.data.Subset(eeg_dataset, range(0, 48))

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(eeg_dataset, [0.6, 0.2, 0.2])

print(len(train_dataset), len(val_dataset), len(test_dataset))

In [ ]:
class EEGEyesOpenCloseClassifier(nn.Module):
    def __init__(self):
        super(EEGEyesOpenCloseClassifier, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, 5, stride=5)
        self.pool1 = nn.AdaptiveAvgPool1d(1)
        self.fc1 = nn.Linear(32, 32)
        self.fc2 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = x.squeeze(1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.sigmoid(x)
        return x


model = EEGEyesOpenCloseClassifier()

In [ ]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.BCELoss()

In [ ]:
lossi = []

model.train()
for epoch in tqdm(range(100)):
    optimizer.zero_grad()
    y_pred = model(train_dataset[0]['window'])

    # if epoch == 0:
    #     print(y_pred)

    loss = criterion(y_pred, train_dataset[0]['labels'])
    lossi.append(loss.item())

    loss.backward()
    optimizer.step()
    
    # if epoch <= 1:
    #     for p in model.parameters():
    #         print(p.grad)
    

plt.plot(lossi)

print(lossi)